<a href="https://colab.research.google.com/github/laurag0/R_thesis_datacleaning/blob/main/PPO%2BOPTUNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mujoco
!pip install --upgrade stable-baselines3
!pip install gymnasium
!pip install tensorboard
!sudo apt-get update
!sudo apt-get install build-essential swig
!pip install git+https://github.com/pybox2d/pybox2d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [5]:
!pip install optuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [6]:
import gymnasium as gym
import numpy as np
import optuna
import pandas as pd
import warnings
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize

In [8]:
import os

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:


warnings.filterwarnings("ignore", category=UserWarning)

def optimize_ppo(trial):
    ent_coef = trial.suggest_float('ent_coef', 0.0, 0.01)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    env = make_vec_env('BipedalWalker-v3', n_envs=8, vec_env_cls=SubprocVecEnv)
    env = VecNormalize(env, norm_obs=True, norm_reward=True)

    log_dir = f"./logs/trial_{trial.number}/"
    os.makedirs(log_dir, exist_ok=True)

    model = PPO("MlpPolicy", env, verbose=1, ent_coef=ent_coef, learning_rate=learning_rate, device='cuda')
    eval_callback = EvalCallback(env, best_model_save_path="./logs/",
                                 log_path="./logs/", eval_freq=50000,
                                 deterministic=True, render=False)

    model.learn(total_timesteps=1e6, callback=eval_callback)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

    env.close()

    return mean_reward

study = optuna.create_study(direction='maximize')
study.optimize(optimize_ppo, n_trials=10)

print(f"Best trial: {study.best_trial.number}")
print(f"Best value: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[I 2024-06-23 00:36:45,596] A new study created in memory with name: no-name-254835fe-16cd-418a-a842-e863f4a2437f


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 473      |
|    ep_rew_mean     | -113     |
| time/              |          |
|    fps             | 787      |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 16384    |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 174       |
|    ep_rew_mean          | -110      |
| time/                   |           |
|    fps                  | 554       |
|    iterations           | 2         |
|    time_elapsed         | 59        |
|    total_timesteps      | 32768     |
| train/                  |           |
|    approx_kl            | 0.6542837 |
|    clip_fraction        | 0.712     |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.94     |
|    explained_variance   | 0.0992    |
|    learning_rate        | 0.00909   |
|  

[I 2024-06-23 01:16:46,536] Trial 0 finished with value: 26.982847900000007 and parameters: {'ent_coef': 0.007772692650983789, 'learning_rate': 0.009093275252702973}. Best is trial 0 with value: 26.982847900000007.


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 364      |
|    ep_rew_mean     | -110     |
| time/              |          |
|    fps             | 769      |
|    iterations      | 1        |
|    time_elapsed    | 21       |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 437         |
|    ep_rew_mean          | -112        |
| time/                   |             |
|    fps                  | 569         |
|    iterations           | 2           |
|    time_elapsed         | 57          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.098176286 |
|    clip_fraction        | 0.545       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.82       |
|    explained_variance   | -0.0118     |
|    learning

[I 2024-06-23 01:57:04,765] Trial 1 finished with value: 202.0735762 and parameters: {'ent_coef': 0.0034378618194220445, 'learning_rate': 0.00587289613391228}. Best is trial 1 with value: 202.0735762.


...

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 501      |
|    ep_rew_mean     | -113     |
| time/              |          |
|    fps             | 823      |
|    iterations      | 1        |
|    time_elapsed    | 19       |
|    total_timesteps | 16384    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 521        |
|    ep_rew_mean          | -113       |
| time/                   |            |
|    fps                  | 580        |
|    iterations           | 2          |
|    time_elapsed         | 56         |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.01987842 |
|    clip_fraction        | 0.27       |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.68      |
|    explained_variance   | -0.00789   |
|    learning_rate        | 

[I 2024-06-23 06:30:23,140] Trial 8 finished with value: 52.7392054 and parameters: {'ent_coef': 0.000516320874494599, 'learning_rate': 0.003382385094724479}. Best is trial 3 with value: 294.4061788.


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 447      |
|    ep_rew_mean     | -110     |
| time/              |          |
|    fps             | 783      |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 16384    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 492        |
|    ep_rew_mean          | -108       |
| time/                   |            |
|    fps                  | 560        |
|    iterations           | 2          |
|    time_elapsed         | 58         |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.08322448 |
|    clip_fraction        | 0.517      |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.76      |
|    explained_variance   | -0.0593    |
|    learning_rate        | 

[I 2024-06-23 07:09:18,394] Trial 9 finished with value: 177.2577788 and parameters: {'ent_coef': 0.00010236191189795685, 'learning_rate': 0.006309197756258059}. Best is trial 3 with value: 294.4061788.


Best trial: 3
Best value: 294.4061788
Best parameters: {'ent_coef': 0.009265413909035564, 'learning_rate': 0.0004945252392340405}
